### Input

In [ ]:
from pathlib import Path
TXT0 = Path('flights.txt').read_text()

### Parse

In [ ]:
TXT = []
for line in TXT0.split("\n"):
    if len(line) == 0:
        continue
    if line[0] == "#":
        continue
    TXT.append(line)

In [ ]:
TXT = [x.strip(" ") for x in TXT]

In [ ]:
trips = [x.split(" ") for x in TXT]
trips[:6]

In [ ]:
import copy, tqdm

In [ ]:
class ListCities:
    def __init__(self):
        self.list_str = set()

    def contains(self, a):
        if type(a) == str:
            return a in self.list_str
        elif type(a) == list:
            return " ".join(a) in self.list_str

    def add(self, a):
        if type(a) == str:
            self.list_str.add(a)
        elif type(a) == list:
            self.list_str.add(" ".join(a))
            

    def remove_candidate_head(self, a):
        if type(a) == str:
            a = a.split(" ")
        assert type(a) == list
        
        for k in range(1, 4):
            if k > len(a):
                break
            candidate = " ".join(a[:k])
            if candidate in self.list_str:
                return [candidate, a[k:]]
        return [None, a]

In [ ]:
def trip_to_extract(trip, list_cities):
    trip = copy.copy(trip)
    if len(trip) == 0:
        return []
    
    cities_on_trip = []
    city, rest = list_cities.remove_candidate_head(trip)
    # print(rest)
    if city is None:
        e = 1
        while True:
            if e > len(trip):
                break
            candidate = trip[:e]
            rest = trip[e:]
            if list_cities.contains(candidate):
                cities_on_trip.append(" ".join(candidate))
                break
            else:
                print(candidate, " ========= ", rest)
                approve = input()
                if approve == "y":
                    list_cities.add(candidate)
                    cities_on_trip.append(" ".join(candidate))
                    break
                else:
                    e = e+1

    else:
        cities_on_trip.append(city)
    
    
    cities_rest = trip_to_extract(rest, list_cities)
    # print("cities_rest = ", cities_rest)
    return cities_on_trip + cities_rest
    


In [ ]:
L = ListCities()

Enter `y` to approve a given breakdown

In [ ]:
trips_formatted = []
for trip in tqdm.tqdm_notebook(trips):
    trips_formatted.append(trip_to_extract(trip, L))

In [ ]:
import pandas as pd

In [ ]:
trips_broken = []
for trip in trips_formatted:
    for d, a in zip(trip, trip[1:]):
        trips_broken.append([d, a])

In [ ]:
DF = pd.DataFrame(trips_broken, columns = ["departure", "arrival"])

In [ ]:
DF.to_csv("flights.csv")

### Analyze

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
DF = pd.read_csv("flights.csv")
DF = DF.loc[:, ["departure", "arrival"]]

In [ ]:
cities_trips = np.concatenate([DF.departure.values, DF.arrival.values])
DCities = pd.DataFrame(cities_trips)
DCities["c"] = 1
DCities.groupby(0).count().sort_values("c").tail(10)